# Analysis for first sky scans from Alto Chorillos
JCH - June 2023


In [ ]:
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

rc('figure',figsize=(20,12))
rc('font',size=12)

from scipy.signal import medfilt
from scipy.interpolate import interp1d
import glob

from qubicpack.qubicfp import qubicfp
from qubic import fibtools as ft

from importlib import reload
import healpy as hp
from datetime import datetime

from astropy.visualization import astropy_mpl_style, quantity_support
# plt.style.use(astropy_mpl_style)
quantity_support()
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_moon, get_sun, get_body


import sys
reps = ['/Users/hamilton/Python/GitQubic/qubic/qubic/scripts/DiversJC/CalibSalta/']
for rep in reps:     
    if rep not in sys.path:
        sys.path.append(rep)
import fitting as fit
import time_domain_tools as tdt


### Files (from Giuseppe D'Alessandro): 
- 2023-05-29_01.45.22__SkyScan-2.5V-DomeOpened-Night
- 2023-05-29_15.21.00__SkyScan-2V-DomeOpened-Day
- 2023-05-29_11.20.47__SkyScan-2.5V-DomeOpened-Day
- 2023-05-28_16.24.20__SkyScan-2.75V-DomeOpened
- 2023-05-28_12.32.03__SkyScan-2.25V-DomeOpened
- 2023-05-27_16.28.40__SkyScan-2.5V-DomeOpened-Daytime

### Copy them: 
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-29/2023-05-29_01.45.22__SkyScan-2.5V-DomeOpened-Night /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-29/ \
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-29/2023-05-29_15.21.00__SkyScan-2V-DomeOpened-Day /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-29/ \
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-29/2023-05-29_11.20.47__SkyScan-2.5V-DomeOpened-Day /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-29/ \
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-28/2023-05-28_16.24.20__SkyScan-2.75V-DomeOpened /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-28/ \
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-28/2023-05-28_12.32.03__SkyScan-2.25V-DomeOpened /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-28/ \
rsync -avzu cca:/sps/qubic/Data/Calib-TD/2023-05-27/2023-05-27_16.28.40__SkyScan-2.5V-DomeOpened-Daytime /Volumes/HD\ JCH\ SSD/Qubic/QubicData/CommissioningTD/2023-05-27/

In [ ]:
rc('figure',figsize=(20,12))
rc('font',size=12)

DirData = '/Volumes/HD JCH SSD/Qubic/QubicData/CommissioningTD/'

files = [
    '2023-05-29_01.45.22__SkyScan-2.5V-DomeOpened-Night', 
    '2023-05-29_15.21.00__SkyScan-2V-DomeOpened-Day',     
    '2023-05-29_11.20.47__SkyScan-2.5V-DomeOpened-Day',   
    '2023-05-28_16.24.20__SkyScan-2.75V-DomeOpened',      
    '2023-05-28_12.32.03__SkyScan-2.25V-DomeOpened',      
    '2023-05-27_16.28.40__SkyScan-2.5V-DomeOpened-Daytime'
]


allqs = []

### Create a QubicSampling object from these
from qubic.samplings import QubicSampling
    
azqubic = 168.5
##### -24.18629364167771, -66.478206557511
AltoChorrillos = EarthLocation(lat=-24.18629364167771*u.deg, lon=-66.478206557511*u.deg, height=4890*u.m)
####utcoffset = -3*u.hour  # Eastern Daylight Time

### Read one file
for file in files:
    day = file[:10]
    print()
    print("Reading:")
    print(day, file)

    thedata = DirData + '/' + day +'/' + file
    ### Read data
    a = qubicfp()
    a.read_qubicstudio_dataset(thedata)

    thk = a.timeaxis(datatype='platform')
    az = a.azimuth()
    el = a.elevation()

    ok = el > 30
    thk=thk[ok]
    az = az[ok]
    el = el[ok]

    date_obs = str(datetime.utcfromtimestamp(thk[0]))
    print('Observations started at: {} UTC'.format(date_obs))

    qs = QubicSampling(azimuth=az + azqubic, elevation=el, time=thk,
                       #period=np.median(thk[1:]-thk[:-1]), 
                       date_obs=date_obs, longitude=float(AltoChorrillos.lon/u.deg), latitude = float(AltoChorrillos.lat/u.deg))
    allqs.append(qs)






In [ ]:
rc('figure',figsize=(20,15))
rc('font',size=12)

fig = figure()
ax = fig.add_subplot(1,1,1,projection='mollweide')
grid()
title('Equatorial')
for i, theqs in enumerate(allqs):

    phi = np.radians(((theqs.equatorial[:,0]+180) % 360)-180)
    theta = np.radians(theqs.equatorial[:,1])

    plot(phi, theta ,'o', label=files[i])
    xlabel('RA from QS (deg)')
    ylabel('DEC from QS (deg)')
legend(fontsize=12)




fig = figure()
ax = fig.add_subplot(1,1,1,projection='mollweide')
grid()
title('Galactic')
for i, theqs in enumerate(allqs):

    phi = np.radians(((theqs.galactic[:,0]+180) % 360)-180)
    theta = np.radians(theqs.galactic[:,1])

    plot(phi, theta ,'o', label=files[i])
    xlabel('Galactic l (deg)')
    ylabel('Galactic b (deg)')
legend(fontsize=12)

In [ ]:
qs[1]